In [61]:
from keras.models import load_model
from joblib import load

model = load_model('archive/best_lstm_model.keras')
scalar_x = load('archive/x_scalar.save')
scalar_y = load('archive/y_scalar.save')

/home/farhaan/miniconda3/envs/py310/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [62]:
import numpy as np

sequence_length = 5
testing = False

if testing:
    print(f"Give atleast {sequence_length} days of data to help you! (This data will be automatically gathered from the evCharger/smartPlugs)")
    charger_company = int(input("Charger Manufacturer [(same as car manufacturer -> 1)/(different manufacturer -> 0)]: "))
    charger_type = int(input("Charger Type [(fast -> 1)/(slow -> 0)]: "))
    x = []
    for session in range(sequence_length):
        temp = []
        print(f"\nSession {session+1}:-\n")
        start_hour = int(input("Plug In hour [24 hours format (10pm -> 22)]: "))
        start_day_of_week = int(input("Day in the week [Sunday -> 0, Saturday -> 6]: "))
        if start_day_of_week in [0, 6]:
            is_weekend = 1
        else:
            is_weekend = 0
        duration = int(input("Plug In duration [in minutes]: "))
        demand = float(input("Plug In power consumption [in Kwh]: "))
        temp.append(charger_company)
        temp.append(charger_type)
        temp.append(start_hour)
        temp.append(start_day_of_week)
        temp.append(is_weekend)
        temp.append(duration)
        temp.append(demand)
        x.append(temp)
    x = np.array(x)
else:
    charger_type = 1
    x = np.array(
        [[1, charger_type, 20, 1, 1, 652, 9.4],
        [1, charger_type, 23, 2, 0, 432, 13.2],
        [1, charger_type, 17, 3, 0, 863, 7.9],
        [1, charger_type, 19, 4, 0, 718, 8.2],
        [1, charger_type, 22, 5, 0, 436, 11.3]]
    )

x.shape


(5, 7)

In [63]:
x_scaled = scalar_x.transform(x)

y_pred = model.predict(x_scaled.reshape(1, sequence_length, x.shape[-1]))
pred = scalar_y.inverse_transform(y_pred)
pred_duration, pred_demand = pred[0][0], pred[0][1]
print(pred_duration, pred_demand)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
418.93832 16.286726


In [64]:
if testing:
    print("This data will be automatically gathered from the evCharger/smartPlugs")
    designed_battary_capacity = float(input("Designed EV battary capacity [in Kwh]: "))
    available_battary_capacity = float(input("Available EV battary capacity [in Kwh]: "))
    designed_range = float(input("Designed range of the EV [in Km]: "))
    available_range = float(input("Available range of the EV [in Km]: "))
    rate_of_charge_inKm = float(input("Rate of increase in range per hour [in Km]: "))

else:
    designed_battary_capacity = 68.0
    available_battary_capacity = 7.14
    designed_range = 449.0
    available_range = 47.145
    if charger_type:
        rate_of_charge_in_Km = 71
    else:
        rate_of_charge_in_Km = 4.8
rate_of_charge_in_Kwh = rate_of_charge_in_Km / (designed_range / designed_battary_capacity)
rate_of_charge_in_Km_per_min = rate_of_charge_in_Km / 60
rate_of_charge_in_Kw_per_min = rate_of_charge_in_Kwh / 60

print(rate_of_charge_in_Km, rate_of_charge_in_Kwh, rate_of_charge_in_Km_per_min, rate_of_charge_in_Kw_per_min)
    

71 10.752783964365257 1.1833333333333333 0.17921306607275428


![Time of Use (Ontario)](time-of-use.png)

In [65]:
from datetime import datetime, timedelta

weekend_pricing = {
    "winter": "off-peak",
    "summer": "off-peak"
}
time_of_use = {
    "winter": {
        "off-peak": {
            "cost": 8.7,
            "time-period": [19, 24, 0, 7]
        },
        "mid-peak": {
            "cost": 12.2,
            "time-period": [11, 17]
        },
        "on-peak": {
            "cost": 18.2,
            "time-period": [7, 11, 17, 19]
        }
    },
    "summer": {
        "off-peak": {
            "cost": 8.7,
            "time-period": [19, 24, 0, 7]
        },
        "mid-peak": {
            "cost": 12.2,
            "time-period": [7, 11, 17, 19]
        },
        "on-peak": {
            "cost": 18.2,
            "time-period": [11, 17]
        }
    }
}

def time_of_use_to_minutes(time_of_use):
    for season in time_of_use:
        for peak_type in time_of_use[season]:
            time_of_use[season][peak_type]["time-period"] = [hour * 60 for hour in time_of_use[season][peak_type]["time-period"]]
    return time_of_use

if testing:
    plugin_datetime = input("Plugin Date & Time [in (yyyy-mm-dd:hh:mm) (24hrs)]: ")
else:
    plugin_datetime = "2024-08-30:17:20"

date_format = "%Y-%m-%d:%H:%M"
date_obj = datetime.strptime(plugin_datetime, date_format)

time_of_use = time_of_use_to_minutes(time_of_use)
plugin_time = date_obj.hour * 60 + date_obj.minute
plugin_season = "summer" if date_obj.month in range(5, 11) else "winter"
is_weekend = date_obj.weekday() >= 5

In [66]:
from math import ceil

def automatic_slot_finder(time_of_use, plugin_time, plugin_season, pred_duration, is_weekend, date_obj):
    time_slot = {}
    while True:
        for peak_type in time_of_use[plugin_season]:
            time_period = time_of_use[plugin_season][peak_type]["time-period"]
            cost = time_of_use[plugin_season][peak_type]["cost"]
            for start_minute, end_minute in zip(time_period[::2], time_period[1::2]):
                if start_minute <= plugin_time <= end_minute:
                    slot_duration = end_minute - plugin_time
                    actual_duration = min(slot_duration, ceil(pred_duration))
                    if is_weekend:
                        cost = time_of_use[plugin_season][weekend_pricing[plugin_season]]['cost']
                    time_slot[f"slot-{len(time_slot)+1}"] = [[plugin_time, plugin_time + actual_duration], cost]
                    pred_duration -= actual_duration
                    plugin_time += actual_duration
                    if pred_duration <= 0:
                        return time_slot
                    if plugin_time == 24 * 60:
                        plugin_time = 0
                        date_obj += timedelta(days=1)
                        is_weekend = date_obj.weekday() >= 5
            
time_slot = automatic_slot_finder(time_of_use, plugin_time, plugin_season, pred_duration, is_weekend, date_obj)
print(time_slot)

{'slot-1': [[1040, 1140], 12.2], 'slot-2': [[1140, 1440], 8.7], 'slot-3': [[0, 19], 8.7]}


In [67]:
def schedular_helper(time_slot, time_from_pred_demand, is_minimum_range=False, og_time_slot=None):
    final_time_slot = {}
    final_cost = 0
    remaining_time_slot = time_slot
    if is_minimum_range:
        time_slot = og_time_slot
    for slot, (time_period, cost) in time_slot.items():
        start_minute, end_minute = time_period
        tot_time_in_slot = end_minute - start_minute
        actual_time = min(tot_time_in_slot, time_from_pred_demand)
        time_key = slot+"-e" if is_minimum_range else slot
        final_time_slot[time_key] = [start_minute, start_minute + actual_time]
        final_cost += (cost / 60) * actual_time
        time_from_pred_demand -= actual_time
        if is_minimum_range:
            remaining_time_slot.pop(slot)
        if time_from_pred_demand <= 0:
            if is_minimum_range:
                remaining_time_slot[slot] = [[start_minute + actual_time, end_minute], cost]
                return final_time_slot, final_cost, remaining_time_slot
            return final_time_slot, final_cost

def automatic_schedular(time_slot, pred_demand, rate_of_charge_in_Km_per_min, rate_of_charge_in_Kw_per_min, designed_battary_capacity, available_battary_capacity, designed_range, available_range, minimum_required_range=100):
    sorted_time_slot = dict(sorted(time_slot.items(), key=lambda item: item[1][1]))
    time_from_pred_demand = ceil(pred_demand / rate_of_charge_in_Kw_per_min)
    distance_from_pred_demand = ceil(time_from_pred_demand * rate_of_charge_in_Km_per_min)
    og_pred_demand = pred_demand
    og_distance_from_pred_demand = distance_from_pred_demand
    og_time_slot = time_slot
    notes = ""
    if pred_demand > (designed_battary_capacity - available_battary_capacity) or distance_from_pred_demand > (designed_range - available_range):
        pred_demand = designed_battary_capacity - available_battary_capacity
        distance_from_pred_demand = designed_range - available_range
        time_from_pred_demand = ceil(pred_demand / rate_of_charge_in_Kw_per_min)
        distance_from_pred_demand = ceil(time_from_pred_demand * rate_of_charge_in_Km_per_min)
        final_time_slot_sorted, final_cost_sorted = schedular_helper(sorted_time_slot, time_from_pred_demand)
        notes = f"Even though the model predicted the required demand of:{og_pred_demand} [in Kwh] and distance of:{og_distance_from_pred_demand} [in Km] from your past changing pattern, due to the avilable energy in the battery, the demand and the distance is changed as {pred_demand} [in Kwh] and {distance_from_pred_demand} [in Km] respectivily"
    elif available_range < minimum_required_range:
        minimum_required_duration = ceil((minimum_required_range - available_range) / rate_of_charge_in_Km_per_min)
        minimum_time_slot_sorted, minimum_cost_sorted, sorted_time_slot = schedular_helper(sorted_time_slot, minimum_required_duration, is_minimum_range = True, og_time_slot=time_slot)
        remaining_time_from_pred_demand = time_from_pred_demand  - minimum_required_duration
        remaining_time_slot_sorted, remaining_cost_sorted = schedular_helper(sorted_time_slot, remaining_time_from_pred_demand)
        final_cost_sorted = minimum_cost_sorted + remaining_cost_sorted
        final_time_slot_sorted = minimum_time_slot_sorted | remaining_time_slot_sorted
        notes = f"Since the battery range is below {minimum_required_range} Km, the EV is charged for {minimum_required_duration} minutes to have the minimum emergency energy and after that the optimization starts"
    else:
        final_time_slot_sorted, final_cost_sorted = schedular_helper(sorted_time_slot, time_from_pred_demand)
        notes = "The battery is charged in an optimal way"

    final_time_slot, final_cost = schedular_helper(og_time_slot, time_from_pred_demand)
    return notes, time_from_pred_demand, distance_from_pred_demand, final_time_slot, final_cost, final_time_slot_sorted, final_cost_sorted


In [68]:
notes, time_from_pred_demand, distance_from_pred_demand, final_time_slot, final_cost, final_time_slot_sorted, final_cost_sorted = automatic_schedular(
    time_slot, pred_demand, rate_of_charge_in_Km_per_min, rate_of_charge_in_Kw_per_min, designed_battary_capacity, 
    available_battary_capacity, designed_range, available_range, minimum_required_range=100)
print(f"Notes: {notes}")
print(f"predicted plugIn duration [in Minutes]: {pred_duration} and predicted demand [in Kwh]: {pred_demand}")
print(f"Time slot without optimization [in Minutes]: {final_time_slot} and its cost ${final_cost/100}")
print(f"Time slot with optimization [in Minutes]: {final_time_slot_sorted} and its cost ${final_cost_sorted/100}")
print(f"predicted charging time [in Minutes]: {time_from_pred_demand} and predicted charging distance [in Km]: {distance_from_pred_demand}")
print(f"Total savings in this charging session: {(final_cost - final_cost_sorted)/100}")

Notes: Since the battery range is below 100 Km, the EV is charged for 45 minutes to have the minimum emergency energy and after that the optimization starts
predicted plugIn duration [in Minutes]: 418.9383239746094 and predicted demand [in Kwh]: 16.286725997924805
Time slot without optimization [in Minutes]: {'slot-1': [1040, 1131]} and its cost $0.1850333333333333
Time slot with optimization [in Minutes]: {'slot-1-e': [1040, 1085], 'slot-2': [1140, 1186]} and its cost $0.15819999999999998
predicted charging time [in Minutes]: 91 and predicted charging distance [in Km]: 108
Total savings in this chsrging session: 0.026833333333333317
